In [1]:
import os
os.chdir("../")
%pwd

'c:\\Users\\roshi\\OneDrive\\Desktop\\Git-1\\Text_Summarization_NLP'

In [2]:
from src.text_summarization.constants import *
from src.text_summarization.utils.common import read_yaml, create_directories
from datasets import load_dataset, load_from_disk

from transformers import (TrainingArguments,Trainer, 
                          DataCollatorForSeq2Seq, 
                          AutoModelForSeq2SeqLM, AutoTokenizer)
import torch


c:\Users\roshi\OneDrive\Desktop\Git-1\Text_Summarization_NLP\.text-summ-venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int  
    warmup_steps: int 
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float 
    gradient_accumulation_steps: int

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = Config_File_Path,
        params_filepath = Params_File_Path):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir= config.root_dir,
            data_path= config.data_path,
            model_ckpt= config.model_ckpt,
            num_train_epochs= params.num_train_epochs,
            warmup_steps= params.warmup_steps,
            per_device_train_batch_size= params.per_device_train_batch_size,
            per_device_eval_batch_size= params.per_device_eval_batch_size,
            weight_decay= params.weight_decay,
            logging_steps= params.logging_steps,
            evaluation_strategy= params.evaluation_strategy,
            eval_steps= params.eval_steps,
            save_steps= params.save_steps,
            gradient_accumulation_steps= params.gradient_accumulation_steps
        )

        return model_trainer_config

In [5]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collector = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        #loading data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir='pegasus-samsum', 
            num_train_epochs= self.config.num_train_epochs, 
            warmup_steps= self.config.warmup_steps,
            per_device_train_batch_size= self.config.per_device_train_batch_size,
            per_device_eval_batch_size= self.config.per_device_eval_batch_size,
            weight_decay= self.config.weight_decay, 
            logging_steps= self.config.logging_steps,
            evaluation_strategy= self.config.evaluation_strategy,
            eval_steps= self.config.eval_steps, 
            save_steps= self.config.save_steps,
            gradient_accumulation_steps= self.config.gradient_accumulation_steps
        )

        # Train model
        trainer = Trainer(model= model_pegasus, args= trainer_args,
                          data_collator=seq2seq_data_collector,
                          tokenizer=tokenizer,
                          train_dataset=dataset_samsum_pt["train"],
                          eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        # Save Model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))

        # Save Tokenizer
        tokenizer.save_pretrianed(os.path.join(self.config.root_dir, "tokenizer"))  

In [6]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config= model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

c:\Users\roshi\OneDrive\Desktop\Git-1\Text_Summarization_NLP\.text-summ-venv\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: '>' not supported between instances of 'str' and 'int'